In [ ]:
import disney_parks

In [1]:
host = "disneydbinstance.c9lxpmnhtw8a.us-east-2.rds.amazonaws.com"
dbname = "DisneyDB"
username = "chrisArnold1228"
password = "Simba49ers!"
port = 3306
weatherAPI = "686f48c707798f004bf424ed785b2c27"

In [2]:
import pymysql
import re
from datetime import datetime
from pytz import timezone
import pandas as pd
tz = timezone('US/Eastern')

In [ ]:
park_ids = open("parkIds.txt", "r")
string = park_ids.read()
park_list = str.splitlines(string)
ids = [x.split(':')[1] for x in park_list]

conn = pymysql.connect(host, user=username,port=port,passwd=password)
cur = conn.cursor()

for current_id in ids:
    #print(current_id)
    park = disney_parks.Park(current_id)
    current_park_name = park.getParkName().replace("'","")
    query = "select * from DisneyDB.Park where Name = '" + current_park_name + "'"
    park_table = pd.read_sql_query(query, conn)
    if park_table.shape[0] < 1:
        insert_park = "insert into DisneyDB.Park (Name) values ('%s')" %(current_park_name)
        cur.execute(insert_park)
        conn.commit()
        park_table = pd.read_sql_query(query, conn)

    park_db_id = park_table['Id'][0]
    park_hours = park.getTodayParkHours()

    today = park_hours['park_open'].strftime('%Y-%m-%d')
    park_open = park_hours['park_open'].strftime('%I:%M %p') if park_hours['park_open'] is not None else 'None'
    park_close= park_hours['park_close'].strftime('%I:%M %p') if park_hours['park_close'] is not None else 'None'
    emh_open = park_hours['emh_open'].strftime('%I:%M %p') if park_hours['emh_open'] is not None else 'None'
    emh_close = park_hours['emh_close'].strftime('%I:%M %p') if park_hours['emh_close'] is not None else 'None'
    # special_open = park_hours['special_open'].strftime('%I:%M %p') if park_hours['special_open'] is not None else 'None'
    # special_close = park_hours['special_close'].strftime('%I:%M %p') if park_hours['special_close'] is not None else 'None'
    query = "insert into DisneyDB.ParkHours (Date, ParkId, ParkOpen, ParkClose, EMHOpen, EMHClose) values ('%s', %i, '%s','%s','%s','%s')"%(today, park_db_id, park_open, park_close, emh_open, emh_close)
    cur.execute(query)
    conn.commit()


In [ ]:
#that got me the operating park hours
#now we should look at wait times per attraction grouped by park

In [3]:
import disney_parks
import pymysql
import pandas as pd
from disney_attractions import Attraction
from disney_entertainment import Entertainment

conn = pymysql.connect(host, user=username,port=port,passwd=password)
cur = conn.cursor()

p_ids = open("parkIds.txt", "r")
string = p_ids.read()
park_list = str.splitlines(string)
ids = [x.split(':')[-1] for x in park_list]

att_ids = open("attractionIds.txt", "r")
string = att_ids.read()
att_list = str.splitlines(string)
attraction_ids = [x.split(':')[-1] for x in att_list]

ent_ids = open("entertainment.txt", "r")
string = ent_ids.read()
ent_list = str.splitlines(string)
entertainment_ids = [x.split(':')[-1] for x in ent_list]

for current_id in ids:
    tz = timezone('US/Eastern')
    park = disney_parks.Park(current_id)
    current_park_name = park.getParkName().replace("'","")
    print("Working On Park:" + str(current_park_name))

    query = "select * from DisneyDB.Park where Name = '" + current_park_name + "'"
    park_table = pd.read_sql_query(query, conn)
    if park_table.shape[0] < 1:
        insert_park = "insert into DisneyDB.Park (Name) values ('%s')" %(current_park_name)
        cur.execute(insert_park)
        conn.commit()
        park_table = pd.read_sql_query(query, conn)

    park_db_id = park_table['Id'][0]
    if (park_db_id == 10) or (park_db_id == 11):
        tz = timezone('US/Pacific')
    #get our wait times

    time = datetime.now(tz)
    date = time.date()
    time = time.time().strftime("%H:%M")

    wait_times = park.getCurrentWaitTimes()
    for key, wait_dict in wait_times.items():
        is_attraction = False
        is_entertainment = False
        if key in attraction_ids:
            is_attraction = True
            #print("this is an attraction")
        elif key in entertainment_ids:
            is_entertainment = True
            #print("this is entertainment")
        else:
            #print("bad_key")
            #print(wait_dict)
            continue

        #print(key)
        #print(wait_dict)
        #check if that id exists in the ride table
        key = key.strip()
        search_query = "select * from DisneyDB.Ride where Id = " + key
        ride_table = pd.read_sql_query(search_query, conn)
        #print(ride_table.shape)
        if ride_table.shape[0] < 1:
            if is_attraction:
                #print("finding an attraction")
                current_attraction = Attraction(key)
                name = current_attraction.getAttractionName()
                #print(name)
                location = current_attraction.getAncestorLand()

            elif is_entertainment:
                #print("finding an entertainment")
                current_entertainment = Entertainment(key)
                name = current_entertainment.getEntertainmentName()
                #print(name)
                location = current_entertainment.getAncestorLand()

            query = "insert into DisneyDB.Ride (Id, Name, Location, ParkId) values (%i, '%s', '%s', %i)"%(int(key),name,location,park_db_id)
            #print(query)
            cur.execute(query)
            conn.commit()

            ride_table = pd.read_sql_query(search_query, conn)

        db_ride_id = ride_table["Id"][0]


        if 'wait_time' in wait_dict:
            current_wait = wait_dict['wait_time']
            query = "insert into DisneyDB.Ride_Waits_Today (RideId, Date, Time, Wait) values (%i, '%s','%s', %i)"%(db_ride_id, date, time, current_wait)
            cur.execute(query)
            conn.commit()

        current_status = "No Status to Report"
        fastpass_availability = "Not a fastpass attraction"
        if 'status' in wait_dict:
            current_status = wait_dict['status']

        if 'fast_pass_info' in wait_dict:
            fast_pass_dict = wait_dict['fast_pass_info']
            if fast_pass_dict['available']:
                fastpass_availability = "Fastpasses are Available Today"
                if 'startTime' in fast_pass_dict:
                    if fast_pass_dict['startTime'] == "FASTPASS is Not Available":
                        fastpass_availability = "Fastpasses are no longer Available Today"
            else:
                fastpass_availability = "Not a fastpass attraction"

        query = "replace into DisneyDB.Ride_Current_Status (RideId, Status, FastPassAvailable) values (%i, '%s', '%s')"%(db_ride_id, current_status, fastpass_availability)
        cur.execute(query)
        conn.commit()
print("success at " + str(time))

Working On Park:Magic Kingdom Park
Working On Park:Epcot
Working On Park:Disneys Hollywood Studios
Working On Park:Disneys Animal Kingdom Theme Park
Working On Park:Disneyland Park
Working On Park:Disney California Adventure Park
Working On Park:Disneys Typhoon Lagoon Water Park
Working On Park:Disneys Blizzard Beach Water Park
success at 14:12


In [ ]:
# testing disneyland 
current_id =  "80007823"

In [ ]:
import disney_parks
import pymysql
import pandas as pd
from disney_attractions import Attraction
from disney_entertainment import Entertainment
conn = pymysql.connect(host, user=username,port=port,passwd=password)
cur = conn.cursor()
att_ids = open("attractionIds.txt", "r")
string = att_ids.read()
att_list = str.splitlines(string)
attraction_ids = [x.split(':')[-1] for x in att_list]

ent_ids = open("entertainment.txt", "r")
string = ent_ids.read()
ent_list = str.splitlines(string)
entertainment_ids = [x.split(':')[-1] for x in ent_list]

tz = timezone('US/Eastern')
park = disney_parks.Park(current_id)
current_park_name = park.getParkName().replace("'","")
print("Working On Park:" + str(current_park_name))

query = "select * from DisneyDB.Park where Name = '" + current_park_name + "'"
park_table = pd.read_sql_query(query, conn)
if park_table.shape[0] < 1:
    insert_park = "insert into DisneyDB.Park (Name) values ('%s')" %(current_park_name)
    cur.execute(insert_park)
    conn.commit()
    park_table = pd.read_sql_query(query, conn)

park_db_id = park_table['Id'][0]
if (park_db_id == 10) or (park_db_id == 11):
    tz = timezone('US/Pacific')
#get our wait times

time = datetime.now(tz)
date = time.date()
time = time.time().strftime("%H:%M")

wait_times = park.getCurrentWaitTimes()
for key, wait_dict in wait_times.items():
    is_attraction = False
    is_entertainment = False
    if key in attraction_ids:
        is_attraction = True
        #print("this is an attraction")
    elif key in entertainment_ids:
        is_entertainment = True
        print("this is entertainment")
    else:
        #print("bad_key")
        #print(wait_dict)
        continue

    #print(key)
    #print(wait_dict)
    #check if that id exists in the ride table
    key = key.strip()
    search_query = "select * from DisneyDB.Ride where Id = " + key
    ride_table = pd.read_sql_query(search_query, conn)
    #print(ride_table.shape)
    if ride_table.shape[0] < 1:
        if is_attraction:
            print("finding an attraction")
            current_attraction = Attraction(key)
            name = current_attraction.getAttractionName()
            print(name)
            location = current_attraction.getAncestorLand()

        elif is_entertainment:
            print("finding an entertainment")
            current_entertainment = Entertainment(key)
            name = current_entertainment.getEntertainmentName()
            print(name)
            location = current_entertainment.getAncestorLand()

        query = "insert into DisneyDB.Ride (Id, Name, Location, ParkId) values (%i, '%s', '%s', %i)"%(int(key),name,location,park_db_id)
        print(query)
        cur.execute(query)
        conn.commit()

        ride_table = pd.read_sql_query(search_query, conn)

    db_ride_id = ride_table["Id"][0]


    if 'wait_time' in wait_dict:
        current_wait = wait_dict['wait_time']
        print(str(db_ride_id) + " " + str(current_wait))
        #query = "insert into DisneyDB.Ride_Waits_Today (RideId, Date, Time, Wait) values (%i, '%s','%s', %i)"%(db_ride_id, date, time, current_wait)
        #cur.execute(query)
        #conn.commit()

In [ ]:
siteString = "https://touringplans.com//" + "animal-kingdom" + "/attractions.json"

In [ ]:
import requests
r = requests.get(siteString)
jsonData = r.json()

In [ ]:
jsonData

In [ ]:
for item in jsonData:
    print(item['name'])

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import pymysql

locations = [{"name":"Magic Kingdom Park", "linkName":"magic-kingdom"},
            {"name":"EpCot","linkName":"epcot"},
            {"name":"Disneys Hollywood Studios","linkName":"hollywood-studios"},
            {"name":"Disneys Animal Kingdom Theme Park","linkName":"animal-kingdom"}]

conn = pymysql.connect(host, user=username,port=port,passwd=password)

cur = conn.cursor()

for location in locations:
    locationName = location["name"]
    linkName = location["linkName"]
    locationQuery = "select * from DisneyDB.Park where name = '" + locationName +"'"
    df = pd.read_sql_query(locationQuery,conn)
    if len(df.index) == 0:
        insertLocation = "insert into DisneyDB.Park (Name) values ('"+ locationName + "')"
        cur.execute(insertLocation)
        df = pd.read_sql_query(locationQuery,conn)
        conn.commit()
    locationId = df['Id'][0]
    siteString = "https://touringplans.com//" + linkName + "/attractions.json"
    r = requests.get(siteString)
    jsonData = r.json()
    #print(jsonData)
    for ride in jsonData:
        tableName = ride["name"]
        rideName = ride["permalink"]
        #tableName = tableName.replace(" ", "-")
        tableName = tableName.replace("'","")
        tableName = tableName.replace(u"\u2019","")
        tableName = tableName.replace("&", "and")
        tableName = tableName.replace(": "," - ")
        print(rideName)
        attractionString = "https://touringplans.com/"+linkName+"/attractions/"+rideName+".json"
        rideRequest = requests.get(attractionString)
        rideJson = rideRequest.json()
        #print(rideJson["name"] + " opened on " + str(rideJson["opened_on"]))
        rideQuery = "select * from DisneyDB.Ride where name = '"+ tableName + "'"
        rideTable = pd.read_sql_query(rideQuery, conn)
        if len(rideTable.index) == 0:
            insertRide = "insert into DisneyDB.Ride (Name, OpeningDate, Tier, ParkId) values ('" + tableName + "','"+ str(rideJson["opened_on"]) + "','"+ str(rideJson["scope_and_scale_code"])+ "',"+str(locationId)+")"
        else:
            rideId = rideTable['Id'][0]
            insertRide = "update DisneyDB.Ride set OpeningDate='"+ str(rideJson["opened_on"]) + "', Tier = '"+ str(rideJson["scope_and_scale_code"])+ "' where Id = "+ str(rideId)
        cur.execute(insertRide)
        conn.commit()


In [ ]:
#can we get restaurants
import disney_parks
import pymysql
import pandas as pd
from disney_attractions import Attraction
from disney_entertainment import Entertainment

In [ ]:
animal_kingdom = disney_parks.Park('80007834')

In [ ]:
animal_kingdom

In [ ]:
animal_kingdom.getParkData()

In [ ]:
data = animal_kingdom.getParkData()

In [ ]:
data.keys()

In [ ]:
data['webLinks']

In [ ]:
import requests
import json
import sys
from datetime import datetime, timedelta
from disney_auth import get_header
from disney_parks import Park

In [ ]:
import disney_attractions

In [ ]:
attraction = disney_attractions.Attraction("353303")

In [ ]:
import disney_parks
import pymysql
import pandas as pd
from disney_attractions import Attraction
from disney_entertainment import Entertainment
conn = pymysql.connect(host, user=username,port=port,passwd=password)
cur = conn.cursor()
att_ids = open("attractionIds.txt", "r")
string = att_ids.read()
att_list = str.splitlines(string)
attraction_ids = [x.split(':')[-1] for x in att_list]

ent_ids = open("entertainment.txt", "r")
string = ent_ids.read()
ent_list = str.splitlines(string)
entertainment_ids = [x.split(':')[-1] for x in ent_list]



In [ ]:
rides = pd.read_sql_query("select * from DisneyDB.Ride", conn)

In [ ]:
ride_ids = rides['Id']

In [ ]:
list(ride_ids)

In [ ]:
for ride in list(ride_ids):
    if str(ride) in attraction_ids:
#         this_ride = Attraction(str(ride))
#         ride_data = this_ride.getAttractionData()
#         if 'descriptions' in ride_data:
#             print("updating description for " + str(this_ride.getAttractionName()))
#             ride_description = this_ride.getMobileDescription()
            
#             query = 'update DisneyDB.Ride set Description = "%s" where Id = %i'%(ride_description, int(ride))
#             cur.execute(query)
#             conn.commit()
            
        "do some stuff and handle as an attraction"
        
    elif str(ride) in entertainment_ids:
        this_ride = Entertainment(str(ride))
        ride_data = this_ride.getEntertainmentData()
        if 'descriptions' in ride_data:
            print("updating description for " + str(this_ride.getEntertainmentName()))
            ride_description = this_ride.getMobileDescription()
            
            query = 'update DisneyDB.Ride set Description = "%s" where Id = %i'%(ride_description, int(ride))
            cur.execute(query)
            conn.commit()
        

In [ ]:
test_case = Attraction("80010110")

In [ ]:
string = test_case.getMobileDescription()

In [ ]:
data = test_case.getAttractionData()

In [ ]:
data.keys()

In [ ]:
data['fastPass']

In [ ]:
string = data['descriptions']['shortDescriptionMobile']['text'].replace(u"\u201c"," ").replace(u"\u201d"," ").replace(u"\u2019"," ").replace(u"\u2014", " ").replace(u"\u2022", " ").replace(u"\u2026", " ").replace(u"\u2013", " ")
string = string.replace(u"\u2018", " ").replace(u"\u2122", " ")
string = string.replace('"', "'")
string = re.sub("<.?a.*?>","",string)

In [ ]:
import re

In [ ]:
re.findall("<a.*>.*</a>", string)

In [ ]:
strings = re.split("<a.*>.*</a>", string)

In [ ]:
strings

In [ ]:
replacement = re.findall("<a.*>.*</a>", string)

In [ ]:
found_occurance = replacement[0]

In [ ]:
new_string = re.sub("<.*?>","",found_occurance)

In [ ]:
new_string

In [ ]:
bad_string = re.findall("<a.*>.*</a?>", string)
if len(bad_string) > 1:
    new_string = ""
    strings = re.split("<a.*>.*</a?>", string)
    for i in range(len(strings) - 1):
        print(i)
        current_occurance = bad_string[i]
        corrected =  
    

In [ ]:
strings = re.sub("<.?a.*?>","",string)

In [ ]:
strings

In [ ]:
string

In [ ]:
magic_kingdom = disney_parks.Park("80007944")

In [ ]:
wait_time_data = magic_kingdom.getCurrentWaitTimes()

In [ ]:
wait_time_data

In [ ]:
authenticaion = get_header()

In [ ]:
from disney_auth import get_header
import requests
import json

In [ ]:
authentication = get_header()

In [ ]:
s = requests.get("https://api.wdpro.disney.go.com/facility-service/theme-parks/{}/wait-times".format("80007944"), headers=authentication)
wait_data = json.loads(s.content)

In [ ]:
wait_data